In [77]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sqlalchemy import create_engine, inspect
import psycopg2 

In [78]:
pres_gdp = pd.read_csv("president_gdp-94.csv")
pres_terms = pd.read_csv("president_terms-94.csv")

In [79]:
pres_gdp.columns = pres_gdp.columns.str.lower()
pres_gdp.columns = pres_gdp.columns.str.replace(" ","_")

In [80]:
pres_terms.columns = pres_terms.columns.str.lower()
pres_terms.columns = pres_terms.columns.str.replace(" ","_")


In [81]:
# PostgreSQL connection parameters
postgres_params = {
    "database": "postgres", # Insert the name of database
    "user": "postgres",     # Insert username if different
    "password": "!QAZ1qaz",         # Insert your password
    "host": "localhost",    # Replace with your PostgreSQL host if not local
    "port": 5432,
    "options": "-c search_path=dbo,public"}

# Establish a connection to PostgreSQL
conn_str = "postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}".format(**postgres_params)
postgres_engine = create_engine(conn_str)

# Write DataFrame to PostgreSQL
pres_gdp.to_sql('pres_gdp', con=postgres_engine, index=False, if_exists='replace')
pres_terms.to_sql('pres_terms', con=postgres_engine, index=False, if_exists='replace')

67

In [82]:
query="""
select * from pres_terms;
"""
pres_terms = pd.read_sql(sql= query, con= postgres_engine)
pres_terms


,start,stop,name,vp,bday,bmonth,byear
0,1789,1797,George Washington,John Adams,22,2,1732
1,1797,1801,John Adams,Thomas Jefferson,30,10,1735
2,1801,1805,Thomas Jefferson,Aaron Burr,13,4,1743
3,1805,1809,Thomas Jefferson,George Clinton,13,4,1743
4,1809,1812,James Madison,George Clinton,16,3,1751
...,...,...,...,...,...,...,...
62,1993,2001,William Clinton,Albert Gore,19,8,1946
63,2001,2009,George Bush,Richard Cheney,12,6,1924
64,2009,2017,Barack Obama,Joseph R. Biden,4,8,1961
65,2017,2021,Donald Trump,Mike Pence,14,6,1946


In [83]:
query="""
select * from pres_gdp;
"""
pres_gdp= pd.read_sql(sql= query, con= postgres_engine)

In [84]:
## EXPAND YEAR OF TERMS PER LINE

# Create a new DataFrame that will contain the expanded rows
expanded_rows = []

# Iterate over each row and duplicate the data for each year between Start and Stop
for i, row in pres_terms.iterrows():
    for year in range(row['start'], row['stop']):
        row_data = row.to_dict()  # Convert row to a dictionary
        row_data['year'] = year   # Add the expanded year to the dictionary
        expanded_rows.append(row_data)


# Create a new DataFrame from the expanded rows
expanded_df = pd.DataFrame(expanded_rows)

# Display the DataFrame (this would save to a file or display the result in your environment)
#expanded_df.to_csv('expanded_president_list.csv', index=False)

#print(expanded_df)
expanded_df.head()

,start,stop,name,vp,bday,bmonth,byear,year
0,1789,1797,George Washington,John Adams,22,2,1732,1789
1,1789,1797,George Washington,John Adams,22,2,1732,1790
2,1789,1797,George Washington,John Adams,22,2,1732,1791
3,1789,1797,George Washington,John Adams,22,2,1732,1792
4,1789,1797,George Washington,John Adams,22,2,1732,1793


In [85]:
pres_terms= pd.merge(expanded_df,pres_gdp, on = "year")

pres_terms.head()

,start,stop,name,vp,bday,bmonth,byear,year,party,first_name,...,nominal_gdp_(million_of_dollars),real_gdp_(millions_of_2017_dollars),gdp_deflator_(index_2017=100),population,nominal_gdp_per_capita_(current_dollars),real_gdp_per_capita_(year_2017_dollars),gdp_growth,gdp_percent_growth,population_growth,population_percent_growth
0,1789,1797,George Washington,John Adams,22,2,1732,1789,Independent,George,...,176,"4,676",3.78,"3,810,000",46.31,"1,229.61",1,0.030,1,0.031
1,1789,1797,George Washington,John Adams,22,2,1732,1790,Independent,George,...,193,"4,975",3.88,"3,929,000",49.1,"1,266.23",1,0.029,1,0.030
2,1789,1797,George Washington,John Adams,22,2,1732,1791,Independent,George,...,210,"5,274",3.98,"4,048,000",51.89,"1,302.85",1,0.042,1,0.030
3,1789,1797,George Washington,John Adams,22,2,1732,1792,Independent,George,...,230,"5,663",4.05,"4,171,000",55.03,"1,357.63",1,0.048,1,0.030
4,1789,1797,George Washington,John Adams,22,2,1732,1793,Independent,George,...,256,"6,113",4.19,"4,297,000",59.67,"1,422.52",1,0.096,1,0.030


In [86]:
## LABEL ENCODE PARTY HERE then make it a numerical, then use it in the Y
## THIS IS WHERE WE WOULD LABEL ENCODE HOROSCOPE AS WELL

# Set index to 'Year' and select columns for categorical_df
categorical_df = pres_terms[['year','name', 'vp','bday', 'party', 'bmonth', 'gdp_growth', 'population_growth']].set_index('year')

# Select numerical columns for numerical_df
numerical_df = pres_terms[['year','gdp_percent_growth', 'population_percent_growth']].set_index('year')

categorical_df.head(3)
# numerical_df


,name,vp,bday,party,bmonth,gdp_growth,population_growth
year,,,,,,,
1789,George Washington,John Adams,22,Independent,2,1,1
1790,George Washington,John Adams,22,Independent,2,1,1
1791,George Washington,John Adams,22,Independent,2,1,1


In [87]:
categorical_df = pd.get_dummies(categorical_df)
categorical_df.head(3)

,bday,bmonth,gdp_growth,population_growth,name_Abraham Lincoln,name_Andrew Jackson,name_Andrew Johnson,name_Barack Obama,name_Benjamin Harrison,name_Calvin Coolidge,...,vp_Walter F. Mondale,vp_William A. Wheeler,vp_office vacant,party_Democrat,party_Democratic-Republican,party_Federalist,party_Independent,party_National Union,party_Republican,party_Whig
year,,,,,,,,,,,,,,,,,,,,,
1789,22,2,1,1,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
1790,22,2,1,1,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
1791,22,2,1,1,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False


In [88]:
# MERGEM THEM THANGS
dummy_df = pd.merge(categorical_df,numerical_df, left_index=True, right_index=True)

dummy_df

,bday,bmonth,gdp_growth,population_growth,name_Abraham Lincoln,name_Andrew Jackson,name_Andrew Johnson,name_Barack Obama,name_Benjamin Harrison,name_Calvin Coolidge,...,vp_office vacant,party_Democrat,party_Democratic-Republican,party_Federalist,party_Independent,party_National Union,party_Republican,party_Whig,gdp_percent_growth,population_percent_growth
year,,,,,,,,,,,,,,,,,,,,,
1789,22,2,1,1,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,0.030,0.031
1790,22,2,1,1,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,0.029,0.030
1791,22,2,1,1,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,0.042,0.030
1792,22,2,1,1,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,0.048,0.030
1793,22,2,1,1,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,0.096,0.030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018,14,6,1,1,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,0.019,0.005
2019,14,6,0,1,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,-0.026,0.004
2020,14,6,1,1,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,0.056,0.002
